In [1]:
import pandas as pd
from datasets import Dataset, Features, Value, Image
import requests
from PIL import Image
import io

In [9]:
df = pd.read_csv("/home/jupyter/repos/marqo-FashionCLIP/curated_datasets/uniqlo_curated/uniqlo_image_ret_raw.csv")
subs = ["item_id","item_url","item_type","item_name","materials","img_url"]
cols = ["metrics"]+[f"input_{x}" for x in subs]+[f"result_{i}_{x}" for i in range(1,25) for x in subs]
df.columns = cols

In [101]:
uniqlo_df = pd.read_csv("/home/jupyter/repos/marqo-FashionCLIP/curated_datasets/uniqlo5k/uniqlo_processed.csv")
uniqlo_dict = uniqlo_df.to_dict('records')
searcher = {}
for item in uniqlo_dict:
    searcher[item['img_url']] = item

## taking only the useful columns: img_urls
subs = ["img_url"]
cols = ["metrics"]+[f"input_{x}" for x in subs]+[f"result_{i}_{x}" for i in range(1,25) for x in subs]
short_df = df[cols]

## creating retrieval images set
retrieval_db = {}
for item in df.values:
    for it in range(1,151,6):
        if isinstance(item[it+5], str) and item[it+5] not in retrieval_db:
            print(item[it+5])
            retrieval_db[item[it+5]] = {
                "id": item[it],
                "item_url": item[it+1],
                "item_type": item[it+2],
                "item_name": item[it+3],
                "materials": item[it+4],
                "img_url": item[it+5]
            }

https://image.uniqlo.com/UQ/ST3/WesternCommon/imagesgoods/469870/sub/goods_469870_sub14_3x4.jpg?width=400
https://image.uniqlo.com/UQ/ST3/WesternCommon/imagesgoods/469862/sub/goods_469862_sub14_3x4.jpg?width=400
https://image.uniqlo.com/UQ/ST3/us/imagesgoods/442063/item/usgoods_32_442063_3x4.jpg?width=400
https://image.uniqlo.com/UQ/ST3/WesternCommon/imagesgoods/469907/sub/goods_469907_sub14_3x4.jpg?width=400
https://image.uniqlo.com/UQ/ST3/WesternCommon/imagesgoods/469863/sub/goods_469863_sub14_3x4.jpg?width=400
https://image.uniqlo.com/UQ/ST3/WesternCommon/imagesgoods/471164/sub/goods_471164_sub17_3x4.jpg?width=400
https://image.uniqlo.com/UQ/ST3/us/imagesgoods/461002/sub/usgoods_461002_sub14_3x4.jpg?width=400
https://image.uniqlo.com/UQ/ST3/us/imagesgoods/469830/sub/usgoods_469830_sub7_3x4.jpg?width=400
https://image.uniqlo.com/UQ/ST3/us/imagesgoods/460318/sub/usgoods_460318_sub14_3x4.jpg?width=600
https://image.uniqlo.com/UQ/ST3/us/imagesgoods/465197/item/usgoods_30_465197_3x4.jpg?

In [105]:
retrieval_df = pd.DataFrame(retrieval_db.values())
retrieval_df['item_ID'] = list(range(len(retrieval_df)))
retrieval_df

,id,item_url,item_type,item_name,materials,img_url,item_ID
0,469870.0,https://www.uniqlo.com/us/en/products/E469870-...,WOMEN/outerwear-and-blazers/ultra-light-down,Ultra Light Down Vest,Face: 100% Nylon/ Filling: Down ( Minimum 90% ...,https://image.uniqlo.com/UQ/ST3/WesternCommon/...,0
1,469862.0,https://www.uniqlo.com/us/en/products/E469862-...,WOMEN/outerwear-and-blazers/ultra-light-down,PUFFTECH Compact Jacket,Face: 100% Nylon/ Filling: 100% Polyester/ Bac...,https://image.uniqlo.com/UQ/ST3/WesternCommon/...,1
2,470142.0,https://www.uniqlo.com/us/en/products/E442063-...,WOMEN/tops/t-shirts,HEATTECH Fleece T-Shirt | Turtleneck,"70% Acrylic, 30% Rayon",https://image.uniqlo.com/UQ/ST3/us/imagesgoods...,2
3,469907.0,https://www.uniqlo.com/us/en/products/E469907-...,WOMEN/outerwear-and-blazers/coats,Single Collar Tweed Coat,"Shell: 50% Wool, 50% Polyester/ Lining: 100% P...",https://image.uniqlo.com/UQ/ST3/WesternCommon/...,3
4,469863.0,https://www.uniqlo.com/us/en/products/E469863-...,WOMEN/outerwear-and-blazers/ultra-light-down,PUFFTECH Compact Vest,Face: 100% Nylon/ Filling: 100% Polyester/ Bac...,https://image.uniqlo.com/UQ/ST3/WesternCommon/...,4
...,...,...,...,...,...,...,...
101,467039.0,https://www.uniqlo.com/us/en/products/E467039-...,MEN/shirts-and-polos/casual-shirts,AIRism Cotton Full Open Polo Shirt,"73% Cotton, 27% Polyester ( 27% Uses Recycled ...",https://image.uniqlo.com/UQ/ST3/us/imagesgoods...,101
102,471031.0,https://www.uniqlo.com/us/en/products/E471031-...,MEN/shirts-and-polos/polo-shirts,DRY-EX Polo Shirt | Kei Nishikori,Body: 100% Polyester ( 50% Uses Recycled Polye...,https://image.uniqlo.com/UQ/ST3/WesternCommon/...,102
103,465194.0,https://www.uniqlo.com/us/en/products/E455388-...,MEN/shirts-and-polos/polo-shirts,DRY Pique Polo Shirt,"Body: 72% Cotton, 28% Polyester ( 28% Uses Rec...",https://image.uniqlo.com/UQ/ST3/WesternCommon/...,103
104,470641.0,https://www.uniqlo.com/us/en/products/E470641-...,MEN/tops/t-shirts,Soft Knitted Fleece T-Shirt,"95% Polyester, 5% Spandex ( 26% Uses Recycled ...",https://image.uniqlo.com/UQ/ST3/WesternCommon/...,104


In [108]:
retrieval_df["image"] = retrieval_df["img_url"].apply(lambda x: Image.open(requests.get(x, stream=True).raw))
# Convert PIL images to bytes
def pil_to_bytes(img):
    buffer = io.BytesIO()
    img.save(buffer, format="PNG")
    return buffer.getvalue()

retrieval_df["image"] = retrieval_df["image"].apply(pil_to_bytes)

In [113]:

# Step 2: Define the Hugging Face Dataset schema
features = Features({
    "id": Value("float64"),
    "item_url": Value("string"),
    "item_type": Value("string"),
    "item_name": Value("string"),
    "materials": Value("string"),
    "img_url": Value("string"),
    "item_ID": Value("int64"),
    "image": Image(),  # This automatically handles the binary image format
})

# Step 3: Convert to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(retrieval_df, features=features)

In [128]:
hf_dataset[0]

{'id': 469870.0,
 'item_url': 'https://www.uniqlo.com/us/en/products/E469870-000/00',
 'item_type': 'WOMEN/outerwear-and-blazers/ultra-light-down',
 'item_name': 'Ultra Light Down Vest',
 'materials': 'Face: 100% Nylon/ Filling: Down ( Minimum 90% Down )/ Back: 100% Nylon',
 'img_url': 'https://image.uniqlo.com/UQ/ST3/WesternCommon/imagesgoods/469870/sub/goods_469870_sub14_3x4.jpg?width=400',
 'item_ID': 0,
 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=400x533>}

In [129]:
# push to HF hub
hf_dataset.push_to_hub("AbhishekSureddy/uniqlo_curated_100", private=True,split="data")

Map:   0%|          | 0/106 [00:00<?, ? examples/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Updating downloaded metadata with the new split.


In [131]:
# loading from HF hub
from datasets import load_dataset
dataset = load_dataset("AbhishekSureddy/uniqlo_curated_100", use_auth_token=True)

Found cached dataset parquet (/home/abhisheksureddy/.cache/huggingface/datasets/AbhishekSureddy___parquet/AbhishekSureddy--uniqlo_curated_100-72efc4f3c56b5e50/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/2 [00:00<?, ?it/s]

In [134]:
dataset['data']

Dataset({
    features: ['id', 'item_url', 'item_type', 'item_name', 'materials', 'img_url', 'item_ID', 'image'],
    num_rows: 106
})